In [1]:
import os
import numpy as np
import pickle as pc
import tensorflow as tf
from utils import *

#from seq2seq import *
from Encoder import *
from Decoder import *

In [2]:
LBNL_DATA_DIR = 'data/LBNL Building 74/lbnlb74electricity.xlsx'
horizon_size = 4
best_loss = 100000000.0
lr = 0.0001
batch_size = 1
max_patience = 7
max_epochs = 100
checkpoint_dir = 'seq2seq_checkpoint'
LOG_INTERVAL = 200
keep_rate = 0.5
enc_unit = 8
dec_unit = 8

attn = 'bah' # 'bah' , 'luong'

debug = False

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
def evaluate():
    # data load
    time, elec = load_data(LBNL_DATA_DIR)
    
    # modify
    elec = missing_value(elec)
    print(elec[40226])
    
    # split data
    train, valid, test = split_dataset(elec)
    
    # build dataset
    test_enc_data, test_dec_data = build_dataset(test, horizon_size)
    print("Build Dataset Finished")
    print("----------------------")
    print("[Test] enc {}\tdec {}".format(len(test_enc_data), len(test_dec_data)))
    
    encoder = Encoder(enc_unit, batch_size, horizon_size, keep_rate)
    decoder = Decoder(dec_unit, batch_size, horizon_size, keep_rate, attn)
    # optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                     encoder=encoder,
                                     decoder=decoder)
  
    ckpt_dir = os.path.join(checkpoint_dir, 'best_lr-{}_hidden-{}_hr-{}'.format(lr, enc_unit, horizon_size))
    latest_ckpt = tf.train.latest_checkpoint(ckpt_dir)    
    print("latest ckpt: {}".format(latest_ckpt))
    checkpoint.restore(latest_ckpt)
    
    loss_obj = tf.keras.losses.MeanSquaredError()
    rmse_obj = tf.keras.metrics.RootMeanSquaredError()
    
    test_batches = batch_iter(test_enc_data, test_dec_data, 1)
    
    test_loss = 0.
    test_rmse = 0.

    for batch_idx, (batch_x, batch_y) in enumerate(test_batches):
        
        batch_enc_input = np.array(list(map(lambda x: list(x), batch_x))) # <eos>:1
        batch_dec_input = np.array(list(map(lambda x: [2] + list(x), batch_y))) # <sos>: 2
        batch_dec_target = np.array(list(map(lambda x: list(x) + [3], batch_y)))
        
        enc_output, enc_state = encoder(batch_enc_input, training=False)
        dec_hidden = enc_state
        
        batch_loss = 0.
        for t in range(batch_dec_target.shape[1]):
            if attn == 'no':
                pred, dec_hidden = decoder(batch_dec_input[:,t], dec_hidden, enc_output, training=False)            
            else:
                pred, dec_hidden, _ = decoder(batch_dec_input[:,t], dec_hidden, enc_output, training=False)
            
            y_true = tf.reshape(batch_dec_target[:,t], (batch_dec_target.shape[0], 1))
            
            loss = loss_obj(y_true, pred)
            batch_loss += tf.reduce_mean(loss)
            
        batch_loss = batch_loss / int(batch_dec_target.shape[1]) #mse        
        batch_rmse = tf.math.sqrt(batch_loss)
        
        test_loss += batch_loss
        test_rmse += batch_rmse
        
    print("test loss: {}\ttest rmse: {}".format(test_loss/(batch_idx+1), test_rmse/(batch_idx+1)))

In [4]:
if __name__ == '__main__':
    evaluate()

/usr/local/lib/python3.6/dist-packages/openpyxl/worksheet/_reader.py:292: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Size of time, elec: 46111	46111
43.899
Size of train, valid, test: 27973	6994	11144
Build Dataset Finished
----------------------
[Test] enc 11136	dec 11136
latest ckpt: seq2seq_checkpoint/best_lr-0.0001_hidden-8_hr-4/best_ckpt_34-34


W1208 11:04:21.604324 140020881848128 base_layer.py:1772] Layer encoder is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W1208 11:04:22.242376 140020881848128 base_layer.py:1772] Layer decoder is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to 

test loss: 911.3013916015625	test rmse: 28.408222198486328
